In [2]:
import torch 
import numpy as np

- aim for a shared phi
- different classifiers for each environment, w_e depends on e
2 phases, sort of:
1. minimize the sum across all environments, the expected loss with env-specific weight vectors
2. fix the phi from step 1. learn a w which is shared among the environments

Why do we want that?
- from the DARE paper (Rosenfeld '22): sharing phi for environments is okay. 
    - even if you "cheat" when you create the featurizer by looking at test env, it doesn't improve the perf. that much
- the bottleneck is actually in learning a _robust w_ .

1. `learn_featurizer` = small neural network, calculate loss carefully
2. `learn_classifier` = calculate average loss


In [4]:
# celebA
H, W = 218, 178

In [3]:
class BasicNet(nn.Module):
    def __init__(self):
        '''
            Initialize the CAFENet by calling the superclass' constructor
            and initializing a linear layer to use in forward().

            Arguments:
                self: This object.
        '''
        super(BasicNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # dim1, dim2 = hw2_utils.IMAGE_DIMS
        # self.fc = torch.nn.Linear(dim1 * dim2, 6, bias=True)
        # check no bias 
        

    def forward(self, x):
        '''
            Computes the network's forward pass on the input tensor.
            Does not apply a softmax or other activation functions.

            Arguments:
                self: This object.
                x: The tensor to compute the forward pass on.
        '''
        x = self.fc1(x)
        return x


NameError: name 'nn' is not defined